# Functions & Co

In [6]:
#include <iostream>
#include <string>
#include <array>
#include <vector>


## My first functions

- To be noticed : strong typing of the input parameters.
- The new type `std::string`.

In [7]:
std::string str( std::array<double,4> qv ) {
  std::string result { "" } ;
  std::string prefix { "[ " } ;  
  for ( auto value : qv ) {
      result += prefix + std::to_string(value);
      prefix = ", ";
  }
  result += " ]" ;
  return result;
}

In [8]:
std::array<double,4> qv { 1., 2., 3., 4. } ;

In [9]:
std::cout<< str(qv) << std::endl ;

[ 1.000000, 2.000000, 3.000000, 4.000000 ]


In [10]:
std::vector<int> v { 2, 4, 6, 8, 10 };

In [11]:
std::cout<< str(v) << std::endl ;

input_line_17:2:14: error: no matching function for call to 'str'
 std::cout<< str(v) << std::endl ;
             ^~~
input_line_13:1:13: note: candidate function not viable: no known conversion from 'std::vector<int>' to 'std::array<double, 4>' for 1st argument
std::string str( std::array<double,4> qv ) {
            ^


Interpreter Error: 

One can provide **another function with the same name**, but different input parameters.

In [12]:
std::string str( std::vector<int> v ) {
  std::string result { "" } ;
  std::string prefix { "[ " } ;  
  for ( auto value : v ) {
      result += prefix + std::to_string(value);
      prefix = ", ";
  }
  result += " ]" ;
  return result;
}

In [13]:
std::cout<< str(v) << std::endl ;

[ 2, 4, 6, 8, 10 ]


When encountering a function call, the compiler will search for the functions with the correct name, and will select  the one whose input parameter types match better. That is called **overload resolution**.


## My first template

- If two functions implementations are the same except some type, one can make this type a template parameter.

In [14]:
template< typename Collection >
std::string str2( Collection v ) {
  std::string result { "" };
  std::string prefix { "[ " };  
  for ( auto value : v ) {
      result += prefix + std::to_string(value);
      prefix = ", ";
  }
  result += " ]";
  return result;
}

In [15]:
std::cout<< str2(qv) << std::endl ;
std::cout<< str2(v) << std::endl ;

[ 1.000000, 2.000000, 3.000000, 4.000000 ]
[ 2, 4, 6, 8, 10 ]


Each time the function is called with a new kind of argument, the compiler try some sort of copy-paste of the implementation with this new type, and succeed or fail.

In [16]:
std::vector<std::vector<int>> vv { { 1, 2 }, { 3, 4, 5 } };

In [17]:
std::cout<< str2(vv) << std::endl;

input_line_20:6:26: error: no matching function for call to 'to_string'
      result += prefix + std::to_string(value);
                         ^~~~~~~~~~~~~~
input_line_23:2:14: note: in instantiation of function template specialization '__cling_N513::str2<std::vector<std::vector<int, std::allocator<int> >,
      std::allocator<std::vector<int, std::allocator<int> > > > >' requested here
 std::cout<< str2(vv) << std::endl;
             ^
/opt/conda/bin/../lib/gcc/x86_64-conda-linux-gnu/7.5.0/../../../../x86_64-conda-linux-gnu/include/c++/7.5.0/bits/basic_string.h:6414:3: note: candidate function not viable: no known conversion from 'std::vector<int, std::allocator<int> >' to 'int' for 1st argument
  to_string(int __val)
  ^
/opt/conda/bin/../lib/gcc/x86_64-conda-linux-gnu/7.5.0/../../../../x86_64-conda-linux-gnu/include/c++/7.5.0/bits/basic_string.h:6419:3: note: candidate function not viable: no known conversion from 'std::vector<int, std::allocator<int> >' to 'unsigned int' for 1st

Interpreter Error: 

*RESTART KERNEL*

In [1]:
#include <iostream>
#include <string>
#include <array>
#include <vector>

In [2]:
template< typename Collection >
std::string str2( Collection v ) {
  std::string result { "" };
  std::string prefix { "[ " };  
  for ( auto value : v ) {
      result += prefix + std::to_string(value);
      prefix = ", ";
  }
  result += " ]";
  return result;
}

In [3]:
using Vint = std::vector<int> ;
using VVint = std::vector<Vint> ;

In [4]:
VVint vv { { 1, 2 }, { 3, 4, 5 } };

I can even overload template functions and ordinary ones.

In [5]:
std::string str2( VVint vv ) {
  std::string result { "" };
  std::string prefix { "[ " };  
  for ( auto v : vv ) {
      result += prefix + str2(v);
      prefix = ", ";
  }
  result += " ]";
  return result;
}

In [6]:
std::cout<< str2(vv) << std::endl;

[ [ 1, 2 ], [ 3, 4, 5 ] ]


Another way may be to overload the case of scalar `int` and `double`: 

In [5]:
std::string str3( int i ) { return std::to_string(i) ; }

In [6]:
std::string str3( double d ) { return std::to_string(d) ; }

In [9]:
template< typename Collection >
std::string str3( Collection coll ) {
  std::string result { "" }, prefix { "[ " };  
  for ( auto elem : coll ) {
      result += prefix + str3(elem);
      prefix = ", ";
  }
  result += " ]";
  return result;
}

In [10]:
VVint vv { { 1, 2 }, { 3, 4, 5 } };
std::cout<< str3(vv) << std::endl;

[ [ 1, 2 ], [ 3, 4, 5 ] ]


The template function `str3()` is *instanciated* twice : once with `Vint` (alias `std::vector<int>`), once with `VVint` (alias `std::vector<std::vector<int>>`). 

## My first references and constants

- The input parameter of the template `str3()`, as any variable, receive a copy by value of the argument, which can be huge of its type is `Vint` or even `VVint`.
- So to avoid the copy and speedup the function call, one may transform the type into a reference : the parameter will then be just a new name for the original object.
- To be noticed : the two `&`.

In [11]:
template< typename Collection >
std::string str3( Collection & vv ) {
  std::string result { "" }, prefix { "[ " };  
  for ( auto & v : vv ) {
      result += prefix + str3(v);
      prefix = ", ";
  }
  return { result + " ]" };
}

In [12]:
std::cout<< str3(vv) << std::endl;

[ [ 1, 2 ], [ 3, 4, 5 ] ]


Yet, if you want to be sure that the function will never change the value of the argument (i.e. if just want to save the copy), then you can also add `const` keyword: 

In [7]:
template< typename Collection >
std::string str3( Collection const & vv ) {
  std::string result { "" }, prefix { "[ " };  
  for ( auto const & v : vv ) {
      result += prefix + str3(v);
      prefix = ", ";
  }
  return { result + " ]" };
}

In [8]:
std::cout<< str3(vv) << std::endl;

[ [ 1, 2 ], [ 3, 4, 5 ] ]


## My First Tuple

When you need to return several results from a function, `std::tuple` can help.

In [1]:
#include <iostream>
#include <tuple>
#include <cmath>

In [2]:
std::tuple<double,double> polar_coordinates( double x, double y ) {
    double norme = std::sqrt(x*x+y*y) ;
    double angle = std::atan(y/x) ;
    return { norme, angle } ;
}

In [3]:
auto result = polar_coordinates(1.,1.) ;
std::cout << std::get<0>(result) << std::endl ;
std::cout << std::get<1>(result) << std::endl ;

1.41421
0.785398


One can directly split the tuple into variables, with the so-called *structured bindings* syntax.

In [4]:
auto [ n, a ] = polar_coordinates(1.,1.) ;
std::cout << n << std::endl ;
std::cout << a << std::endl ;

1.41421
0.785398


# Quizz

What lacks so that the values are not 0 ?

In [3]:
#include <iostream>
#include <array>
#include <random>

In [5]:
std::vector<int> values(5) ;

std::default_random_engine engine ;
std::uniform_int_distribution distrib{0,9} ;

for ( auto value : values ) {
    value = distrib(engine) ;
}

for ( auto value : values ) {
    std::cout << value << " " ;
}

std::cout << std::endl;

0 0 0 0 0 


# Take away

- You can have many functions with the same name, but different input parameters.
- For every call, the compiler will select the right one for you, most of the time...
- You can pass argument by value, by reference, by const reference.
- You can return several results as a tuple.


# Questions ?

© *CNRS 2023*  
*This document was created by David Chamont. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*